# Northern Ireland Tabulated Weekly Deaths

This workbook is used to prepare a simple dataset that tabulates all of the weekly deaths in a single table using the convention of 'YYYY-MM-DD' for the unique date representing the end of the registration week.

## Setup

In [1]:
!pip install plotly
!pip install ipywidgets>=7.6
!pip install openpyxl
!pip install xlrd
!pip install pandas

In [1]:
import pandas as pd

import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio

from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

from lib.utility_functions import local_ca_certificate_file_path, \
mutate_safely, debug_this, Month, download_file, \
remove_timestamp, rename_columns, convert_dtypes_obj_to_strings, \
inner_join_with, add_week_ending_date, convert_column_to_string, \
extract_only_this_year, extract_and_cast_as_int

In [2]:
init_notebook_mode(connected = True)
pio.renderers.default = 'iframe'

In [3]:
debug_enabled = True
preview_only_enabled = False

In [4]:
def debug(item):
    debug_this(debug_enabled, item, preview_only_enabled)

In [16]:
@mutate_safely
def add_year_as_variable(df, year):
    df['Year']=year
    return

In [27]:
def get_columns(df, columns):
    return df[columns]

In [28]:
core_columns = ["Week Ending Date", "Deaths", "Year", "Registration_Week"]

## 2023 Deaths

In [18]:
analysis_end_week = 48

In [19]:
nisra_weekly_deaths_2023_source_url="https://www.nisra.gov.uk/system/files/statistics/Weekly_Deaths%20-%20w%20e%201%20December%202023.xlsx"
raw_nisra_weekly_deaths_2023_dest_filepath=f"data/inbound/raw/nisra/deaths/weekly/weekly_deaths_upto_including_{analysis_end_week}_2023.xlsx"
rows_to_read = analysis_end_week # Two rows fewer than the row number of the last data point.

In [7]:
download_file(nisra_weekly_deaths_2023_source_url, raw_nisra_weekly_deaths_2023_dest_filepath)

downloaded from: https://www.nisra.gov.uk/system/files/statistics/Weekly_Deaths%20-%20w%20e%201%20December%202023.xlsx


In [20]:
df_2023 = pd.read_excel(raw_nisra_weekly_deaths_2023_dest_filepath,  
                                  sheet_name='Table 3', 
                                  skiprows=3, 
                                  nrows=rows_to_read,
                                  usecols='A:N')
debug(df_2023)

,Registration Week,Week Ending (Friday),Antrim & Newtownabbey,Ards & North Down,"Armagh City, Banbridge & Craigavon",Belfast,Causeway Coast & Glens,Derry City & Strabane,Fermanagh & Omagh,Lisburn & Castlereagh,Mid & East Antrim,Mid Ulster,"Newry, Mourne & Down",Total
0,1,2023-01-06,31,43,43,89,39,38,48,31,31,34,47,474
1,2,2023-01-13,41,46,47,97,42,37,37,32,39,45,52,515
2,3,2023-01-20,38,43,36,83,35,32,24,31,29,17,41,409
3,4,2023-01-27,30,36,41,72,39,35,16,28,35,26,35,393
4,5,2023-02-03,24,37,31,77,29,35,20,31,28,17,39,368
5,6,2023-02-10,28,24,30,71,31,21,19,30,26,22,35,337
6,7,2023-02-17,30,36,47,77,30,40,28,22,24,24,44,402
7,8,2023-02-24,26,33,26,52,31,34,25,22,33,29,31,342
8,9,2023-03-03,23,41,35,53,29,16,22,29,21,25,25,319
9,10,2023-03-10,27,34,26,68,35,18,21,17,36,24,28,334


In [30]:
year="2023"

df_2023_refined = df_2023\
.pipe(rename_columns, {"Registration Week":"Registration_Week", "Week Ending (Friday)":"Week_end_Date","Total":f"Deaths"})\
.pipe(convert_dtypes_obj_to_strings)\
.pipe(add_year_as_variable, year=year)\
.pipe(convert_column_to_string, colname="Week_end_Date")\
.pipe(add_week_ending_date, default_date="2020-19-03", existing_week_end_date_col_name='Week_end_Date')\
.pipe(get_columns, columns=core_columns)

df_2023_refined

,Week Ending Date,Deaths,Year,Registration_Week
0,2023-01-06,474,2023,1
1,2023-01-13,515,2023,2
2,2023-01-20,409,2023,3
3,2023-01-27,393,2023,4
4,2023-02-03,368,2023,5
5,2023-02-10,337,2023,6
6,2023-02-17,402,2023,7
7,2023-02-24,342,2023,8
8,2023-03-03,319,2023,9
9,2023-03-10,334,2023,10


## 2022 Deaths

In [23]:
nisra_weekly_deaths_2020_2022_source_url='https://www.nisra.gov.uk/system/files/statistics/Weekly_Deaths%20-%20w%20e%2030%20December%202022_0.xlsx'
raw_nisra_weekly_deaths_2022_dest_filepath='data/inbound/raw/nisra/deaths/weekly/weekly_deaths_december_2022.xlsx'
rows_to_read = 146 # Four rows fewer than the row number of the last data point.

In [11]:
download_file(nisra_weekly_deaths_2020_2022_source_url, raw_nisra_weekly_deaths_2022_dest_filepath)

downloaded from: https://www.nisra.gov.uk/system/files/statistics/Weekly_Deaths%20-%20w%20e%2030%20December%202022_0.xlsx


In [24]:
df_2020_2022 = pd.read_excel(raw_nisra_weekly_deaths_2022_dest_filepath,  
                                  sheet_name='Table 3', 
                                  skiprows=3, 
                                  nrows=rows_to_read,
                                  usecols='A:N')

In [31]:
year="2022"

df_2022_refined = df_2020_2022\
.pipe(extract_only_this_year, year=year, rows_to_read=rows_to_read)\
.pipe(rename_columns, {"Registration Week":"Registration_Week", "Week Ending (Friday)":"Week_end_Date","Total":f"Deaths"})\
.pipe(convert_dtypes_obj_to_strings)\
.pipe(add_year_as_variable, year=year)\
.pipe(convert_column_to_string, colname="Week_end_Date")\
.pipe(add_week_ending_date, default_date="2020-19-03", existing_week_end_date_col_name='Week_end_Date')\
.pipe(get_columns, columns=core_columns)

debug(df_2022_refined)

,Week Ending Date,Deaths,Year,Registration_Week
0,2022-01-07,416,2022,1
1,2022-01-14,445,2022,2
2,2022-01-21,317,2022,3
3,2022-01-28,336,2022,4
4,2022-02-04,363,2022,5
5,2022-02-11,375,2022,6
6,2022-02-18,319,2022,7
7,2022-02-25,327,2022,8
8,2022-03-04,338,2022,9
9,2022-03-11,338,2022,10


## 2015-2021

In [6]:
#https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/Final%20Historical%20Weekly%20Deaths%20by%20Age%20and%20Respiratory%20Deaths%202011-2022.xlsx
#https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/Final%20Historical%20Weekly%20Deaths%20by%20Age%20and%20Respiratory%20Deaths%202011-2021.xlsx
raw_nisra_historical_weekly_deaths_source_url='https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/Final%20Historical%20Weekly%20Deaths%20by%20Age%20and%20Respiratory%20Deaths%202011-2022.xlsx'
raw_nisra_historical_weekly_deaths_dest_filepath='data/inbound/raw/nisra/deaths/weekly/historical_weekly_deaths.xlsx'

In [ ]:
download_file(raw_nisra_historical_weekly_deaths_source_url, raw_nisra_historical_weekly_deaths_dest_filepath)

In [7]:
year_registration_week_count = {
    2015: 53,
    2016: 52,
    2017: 53,
    2018: 52,
    2019: 52,
    2020: 52,
    2021: 52
}

In [46]:
dfs_2015_2021 = {}

for year, registration_weeks in year_registration_week_count.items():
    print(f'Processing {year} which has {registration_weeks} registration weeks...')
    
    df = pd.read_excel(raw_nisra_historical_weekly_deaths_dest_filepath,  
                                  sheet_name=f'Weekly Deaths_{year}', 
                                  skiprows=3, 
                                  nrows=registration_weeks,
                                  usecols='A:D')
    
    dfs_2015_2021[year] = df\
    .pipe(rename_columns, {"Registration Week":"Registration_Week", f"Total Number of Deaths Registered in Week ({year})":f"Deaths"})\
    .pipe(convert_dtypes_obj_to_strings)\
    .pipe(add_year_as_variable, year=year)\
    .pipe(convert_column_to_string, colname="Week Ends (Friday)")\
    .pipe(add_week_ending_date, default_date="2020-19-03", existing_week_end_date_col_name="Week Ends (Friday)")\
    .pipe(extract_and_cast_as_int, column="Registration_Week")\
    .pipe(get_columns, columns=core_columns)

Processing 2015 which has 53 registration weeks...
Processing 2016 which has 52 registration weeks...
Processing 2017 which has 53 registration weeks...
Processing 2018 which has 52 registration weeks...
Processing 2019 which has 52 registration weeks...
Processing 2020 which has 52 registration weeks...
Processing 2021 which has 52 registration weeks...


In [49]:
df_2015_2021 = pd.concat(dfs_2015_2021.values(), ignore_index=True)
df_2015_2022 = pd.concat([df_2015_2021, df_2022_refined], ignore_index=True)
df_2015_2023 = pd.concat([df_2015_2022,df_2023_refined], ignore_index=True)

debug(df_2015_2023)

,Week Ending Date,Deaths,Year,Registration_Week
0,2015-01-02,319,2015,1
1,2015-01-09,374,2015,2
2,2015-01-16,383,2015,3
3,2015-01-23,397,2015,4
4,2015-01-30,374,2015,5
...,...,...,...,...
461,2023-11-03,341,2023,44
462,2023-11-10,386,2023,45
463,2023-11-17,364,2023,46
464,2023-11-24,404,2023,47


In [50]:
df_2015_2023.to_pickle(f'data/outbound/deaths/weekly/AllWeeklyDeaths2015Week1To2023Week{analysis_end_week}.pkl')

In [ ]:
#TODO - Let's add in a dataset for the Year MYE for population.
# Let's join that dataset to the one above, joining on the Year column and then we can calculate the number of weekly deaths per 100k of the population?